# 1. Library import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import pyapep.isofit as isof
import pyapep.simide as simi

import matplotlib.pyplot as plt
import numpy as np

parameters = {'axes.labelsize': 17,
              'xtick.labelsize': 16,
              'ytick.labelsize': 16,
          'axes.titlesize': 20}
plt.rcParams.update(parameters)
plt.rcParams['font.family'] = 'arial'

from utils import *

# 2. Simulation results import

In [2]:
#File import
Data_exp = pd.read_csv(f'RawData/2.StepwiseIsoFit.csv', # encoding='euc-kr'
                                header = 0)

Data_exp.fillna('NaN', inplace=True)
Data_exp.columns

# ==> 선별된 흡착제 3153개

Index(['Adsorbent', 'CO2_0.1bar', 'CO2_2.67bar', 'CH4_0.2bar', 'CH4_5.33bar',
       'CO2_wc', 'CH4_wc', 'CO2_sc', 'CO2_3p', 'CH4_3p', 'CO2_3p_sim',
       'CH4_3p_sim', 'CH4_Isotherm_1', 'par_1_CH4', 'par_2_CH4',
       'Acc_CH4(-R2)', 'CO2_Isotherm_1', 'par_1_CO2', 'par_2_CO2',
       'Acc_CO2(-R2)', 'CO2_4p', 'CH4_4p', 'CO2_4p_sim', 'CH4_4p_sim',
       'CH4_Isotherm_2', 'par_CH4_2', 'Acc_CH4(-R2)_2', 'CO2_Isotherm_2',
       'par_CO2_2', 'Acc_CO2(-R2)_2', 'CO2_5p', 'CO2_5p_sim', 'CO2_Isotherm_3',
       'par_CO2_3', 'Acc_CO2(-R2)_3', 'CH4_Isotherm_3', 'Density',
       'Unnamed: 37'],
      dtype='object')

# 3. PSA process simulation

## 3.1 PSA process simulation

In [ ]:
# process variable setting 
PSA_config = 'PSA'          # PSA configuration 
P_feed = 8                  # Feed presure (bar)
T_feed = 323.15             # Feed temperature (K)
y_feed = [0.33, 0.67]       # Feed mole fraction (mol/mol) (CO2 / CH4)
P_high1 = 8  # P_high1: Operating pressure for PSA and VSA, or the first column operating pressure for 'cas'
P_high2 = 8  # P_high2: Operating pressure for the second column in 'cas'
P_low = 1    # Low pressure (bar) ==> If pressure is set below 1 bar, VPSA simulation becomes possible


# Perform process simulation ==> Save the mole fraction of tail gas
P_high1_range=range(2,21)

empty = pd.DataFrame()
for P_high1 in P_high1_range:    
    res_list = []
    for j in range(len(Data_exp)):
        print(f'cycle {P_high1} #### {j/len(Data_exp)*100} % #####')
        try:
            target_cof = Data_exp.iloc[j,:]
            
            # Derive isotherms for the two gases
            mix_iso =WhichIsoFunction(target_cof)

            # Derive the mixture isotherm function using IAST (Ideal Adsorbed Solution Theory)
            iso_mix = lambda P,T: isof.IAST(mix_iso, P, T)

            # Ideal PSA process simulation 
            globals()[f'CI_{j}'] = simi.IdealColumn(2, iso_mix, )
            globals()[f'CI_{j}'].feedcond(P_feed, T_feed, y_feed)
            globals()[f'CI_{j}'].opercond(P_high1, P_low)
            
            # Derive Tail gas composition
            x_ext = globals()[f'CI_{j}'].runideal()
                    
        except:
            x_ext = np.array([-2023, -2023])
        
        # save results
        res_list.append(x_ext)    
    empty[f'pu_CO2_PSA_{P_high1}'] = np.array(res_list)[:,0]
    empty[f'pu_CH4_PSA_{P_high1}'] = np.array(res_list)[:,1]

In [8]:
# Calculate recovery for each VSA pressure and save the results to a file
x=y_feed[1] # feed CH4 concentration 

for P_high1 in P_high1_range:    
    CO2 = empty[f'pu_CO2_{PSA_config}_{P_high1}']
    CH4 = empty[f'pu_CH4_{PSA_config}_{P_high1}']
    recovery =1-(CH4*(x-1))/(x*(CH4-1))
    empty[f'Re_CH4_{PSA_config}_{P_high1}']=recovery
empty

empty.to_csv('PSA_process_simulation.csv')

## 3.2 VSA process simulation

In [ ]:
# process variable setting
PSA_config = 'VSA'

P_feed = 8                  # Feed presure (bar)
T_feed = 323.15             # Feed temperature (K)
y_feed = [0.33, 0.67]       # Feed mole fraction (mol/mol) (CO2 / CH4)
P_high1= 8                  

# Perform process simulation ==> Save the mole fraction of tail gas
P_low_range=(0.01,0.04,0.07,0.1,0.3,0.5,0.7,0.9,1)

empty = pd.DataFrame()
for P_low in P_low_range:
    res_list = []
    for j in range(len(Data_exp)):
        print(f'cycle {P_low} #### {j/len(Data_exp)*100} % #####')
        try:
            target_cof = Data_exp.iloc[j,:]
            
            # Derive isotherms for the two gases
            mix_iso =WhichIsoFunction(target_cof)

            # Derive the mixture isotherm function using IAST (Ideal Adsorbed Solution Theory)
            iso_mix = lambda P,T: isof.IAST(mix_iso, P, T)

            # Ideal PSA process simulation 
            globals()[f'CI_{j}'] = simi.IdealColumn(2, iso_mix, )
            globals()[f'CI_{j}'].feedcond(P_feed, T_feed, y_feed)
            globals()[f'CI_{j}'].opercond(P_high1, P_low)
            
            # Derive Tail gas composition
            x_ext = globals()[f'CI_{j}'].runideal()
                    
        except:
            x_ext = np.array([-2023, -2023])
        
        # save results
        res_list.append(x_ext)    
    empty[f'pu_CO2_VSA_{P_low}'] = np.array(res_list)[:,0]
    empty[f'pu_CH4_VSA_{P_low}'] = np.array(res_list)[:,1]


In [31]:
# Calculate recovery for each VSA pressure and save the results to a file
x=y_feed[1] # feed CH4 concentration

for P_low in P_low_range:
    CO2 = empty[f'pu_CO2_{PSA_config}_{P_low}']
    CH4 = empty[f'pu_CH4_{PSA_config}_{P_low}']
    recovery =1-(CH4*(x-1))/(x*(CH4-1))
    empty[f'Re_CH4_{PSA_config}_{P_low}']=recovery

empty.to_csv('VSA_process_simulation.csv')

## 3.3 cascade PSA process simulation 

In [4]:
PSA_data=pd.read_csv('PSA_process_simulation.csv', header=0)

In [ ]:
# process variable setting
PSA_config = 'cas'

P_feed = 8      # Feed presure (bar)
T_feed = 323.15    # Feed temperature (K)
y_feed = [0.33, 0.67] # Feed mole fraction (mol/mol) (CO2 / CH4)
P_low  = 1 # Low pressure (bar)    ==> If the pressure is set below 1 bar, VPSA simulation becomes possible


empty = pd.DataFrame()
for P_high1 in range(2,15):  
    for P_high2 in range(2,15):    
        res_list = []
        for j in range(len(Data_exp)): 
            x_ext1 = [PSA_data.at[j,f'pu_CO2_PSA_{P_high1}'], PSA_data.at[j,f'pu_CH4_PSA_{P_high1}']] # Feed mole fraction (mol/mol) (CO2 / CH4)
            print(f'cycle {P_high1}/{P_high2} #### {j/len(Data_exp)*100} % #################')
            try:
                target_cof = Data_exp.iloc[j,:]
                
                # Derive isotherms for the two gases
                mix_iso =WhichIsoFunction(target_cof)

                # Derive the mixture isotherm function using IAST (Ideal Adsorbed Solution Theory)
                iso_mix = lambda P,T: isof.IAST(mix_iso, P, T)
                
                # Ideal PSA process simulation 
                globals()[f'CI2_{j}'] = simi.IdealColumn(2, iso_mix, )   
                globals()[f'CI2_{j}'].feedcond(P_feed, T_feed, x_ext1)
                globals()[f'CI2_{j}'].opercond(P_high2, P_low) 

                # Derive Tail gas composition
                x_ext2 = globals()[f'CI2_{j}'].runideal()
                        
            except:
                x_ext2 = np.array([-2023, -2023])
                
            # save results
            res_list.append(x_ext2)
        empty[f'pu_CO2_cas_{P_high1}_{P_high2}'] = np.array(res_list)[:,0]
        empty[f'pu_CH4_cas_{P_high1}_{P_high2}'] = np.array(res_list)[:,1]

In [ ]:
# Calculate recovery for each VSA pressure and save the results to a file
x=y_feed[1] # feed CH4 concentration 
for P_high1 in range(2,15):  
    for P_high2 in range(2,15):  
        CO2 = empty[f'pu_CO2_{PSA_config}_{P_high1}_{P_high2}']
        CH4 = empty[f'pu_CH4_{PSA_config}_{P_high1}_{P_high2}']
        recovery =1-(CH4*(x-1))/(x*(CH4-1))
        empty[f'Re_CH4_{PSA_config}_{P_high1}_{P_high2}']=recovery

empty.to_csv('cascade_PSA_process_simulation.csv')

In [42]:
PSA_data=pd.read_csv('PSA_process_simulation.csv', header=0, index_col=0)
VSA_data=pd.read_csv('VSA_process_simulation.csv', header=0, index_col=0 )
cascade_PSA_data=pd.read_csv('cascade_PSA_process_simulation.csv', header=0, index_col=0)

In [43]:
process_simulation_data = pd.concat([Data_exp,PSA_data, VSA_data,cascade_PSA_data], axis=1)
process_simulation_data.to_csv('All_process_simulation_data.csv')

# 4. Economic analysis

In [44]:
# Input parameters for economic evaluation
r = 0.08            # Interest rate (%)
t = 25              # Project lifetime (yr)
porosity = 0.2      # Column porosity (-)
att_ratio = 0.1     # Adsorbent attrition rate (%/yr)

Vacuum_pump_cost= 188094

def Compressor_cost(P):
    if P < 3.14:
        CC = 239533
    elif P < 10:
        CC = 2*239533
    elif P <=20:
        CC = 3*239533
    return CC

def PSA_cost(PSA_config,P_high1,P_high2):
    if PSA_config == 'PSA':
        PSA_cost = 486630 + Compressor_cost(P_high1)
    elif PSA_config == 'VSA':
        PSA_cost = 486630 + Compressor_cost(P_high1)+ Vacuum_pump_cost
    elif PSA_config == 'cas':
        PSA_cost = 486630 + Compressor_cost(P_high1) +(486630+Compressor_cost(P_high2))*(0.4**0.6)  # The second column is expected to be smaller in size
    return PSA_cost

# Calculate the energy consumption of the compressor and vacuum pump (expressed in kWh/year)
def PEnergy(PSA_config,P_high1,P_low,P_high2):
    R = 8.314           # Gas const. (J/mol/K)
    if PSA_config == 'PSA' or PSA_config == 'VSA':
        # Single-stage PSA high-pressure operation
        T_rev_1= 298*(P_high1/1)**(2/7)
        W_rev = 10.12*(7/2)*8.314*(T_rev_1-298.15)/0.8
        
        # Work required under VSA pressure conditions below 1 bar
        T_rev_2 = 298*(P_low/1)**(2/7)
        W_rev += -10.12*(7/2)*8.314*(T_rev_2-298.15)/0.8
        electricity = W_rev/3600000*3600*7920      # unit conversion (J/s to kWh/yr)
    elif PSA_config == 'cas':
        # Work required for first-stage PSA high-pressure operation
        T_rev_1= 298*(P_high1/1)**(2/7)
        W_rev = 10.12*(7/2)*8.314*(T_rev_1-298.15)/0.8
        
        # Work required for second-stage PSA high-pressure operation
        T_rev_2= 298*(P_high2/1)**(2/7)
        W_rev += 10.12*0.4*(7/2)*8.314*(T_rev_2-298.15)/0.8  # Set the tail gas flow rate to 0.4 times the single-stage feed flow rate.
        electricity = W_rev/3600000*3600*7920 
    return electricity

# Econonmic analysis function
def Economic(PSA_config, target, ads_cost, density, P_high1, P_low, P_high2, recovery = None):
    # CAPEX
    equipment = 2891628 + PSA_cost(PSA_config,P_high1,P_high2) # USD
    FCI = equipment/0.428       # Fixed capital investment
    WC = FCI*0.1                # Working capital
    TCI = FCI+WC
    AF = (1-(1/(1+r)**t))/r     # Annualized factor
    EAC = TCI / AF              # Annualized capital cost

    # OPEX
    FC = FCI*0.035                  # Fixed charge
    RM = 27 * 25344                 # Raw material (Grass silage, 12USD/ton)
    Steam = 12 *  1543.44           # Steam (12USD/ton)

    single_column_volume = (0.45/2)**2*3.14*2.25*(816/500)*3    # Volume of a single PSA column (reference)
    if PSA_config == 'cas':
        PSA_vol = single_column_volume*2*1.4        # Column volume (m³) (cascade has 1.4 times greater capacity)
    else:
        PSA_vol = single_column_volume*2            # column volume (m3)
    
    ads_repl = 2                    # Adsorbent replacement (year)
    Ads_cost = ads_cost*density * PSA_vol / ads_repl # Adsorbent cost (USD/yr)

    Elect = 0.075 * (1160250 + PEnergy(PSA_config,P_high1,P_low,P_high2)) # Electricity
    M = FCI*0.02                        # Maintenance
    OL = 3*30000                        # Operating labor (3 person)
    S = OL * 0.15                       # Supervision and support labor
    OS = M * 0.1                        # Operating supplies
    OVHD = (M + OL + S) * 0.2           # Plant overhead cost

    VPC = RM + Steam + Ads_cost + Elect + M + OL + S + OS   # Variable production cost
    TMC = FC+VPC+OVHD                                       # Total manufacturing cost
    GE = OL * 0.15                                          # General expenses
    TPC = TMC + GE                                          # Total production cost

    TAC = EAC + TPC
    if recovery == None:
        if PSA_config == 'PSA':
            recovery = target[f'Re_CH4_PSA_{P_high1}']
        elif PSA_config == 'VSA':
            recovery = target[f'Re_CH4_VSA_{P_low}']
        elif PSA_config == 'cas':
            recovery = target[f'Re_CH4_cas_{P_high1}_{P_high2}']
            
    CH4_prod = recovery * 1706697      # Annual CH4 production (F_f,CH4 = 1706697 kg/yr)
    LC = TAC/CH4_prod            # Levelized cost ($/kg H4)
    return int(EAC), int(TPC), LC

In [ ]:
# Load simulation results
data = pd.read_csv('All_process_simulation_data.csv', index_col=0).fillna('NaN')  # Replace missing values with 'NaN'
data.head()  # Display the first few rows of the DataFrame
data


In [ ]:
data_clean = data[data['Density'] != 'NaN']  # Exclude rows where 'Density' is NaN to avoid errors during MACO calculation
data_clean

### LCOM calculation

In [47]:
# Fixed adsorbent cost

# Save CH4 prices as adsorbent cost changes from 10 to 10^6 USD/kg
adsC_list = np.linspace(1, 6, 6)  # Adsorbent cost range in logarithmic scale
P_high1 = 8
P_low = 1
P_high2 = 8

# Automatically extract economic data for each configuration and pressure condition
for PSA_config in ('PSA', 'VSA', 'cas'):
    if PSA_config == 'PSA':
        for factor in adsC_list:
            adsC = 10**factor
            for P_high1 in range(2, 21):  # Loop over high pressures
                data_clean[f'LC_10^{factor}_{PSA_config}_{P_high1}'] = [
                    Economic(PSA_config, data_clean.iloc[ii, :], adsC, 
                             data_clean.iloc[ii, :]['Density'], P_high1, P_low, P_high2)[-1] 
                    for ii in range(len(data_clean))
                ]

    elif PSA_config == 'VSA':
        P_high1 = 6
        for factor in adsC_list:
            adsC = 10**factor
            for P_low in (0.01, 0.04, 0.07, 0.1, 0.3, 0.5, 0.7, 0.9, 1):  # Loop over low pressures
                data_clean[f'LC_10^{factor}_{PSA_config}_{P_low}'] = [
                    Economic(PSA_config, data_clean.iloc[ii, :], adsC, 
                             data_clean.iloc[ii, :]['Density'], P_high1, P_low, P_high2)[-1] 
                    for ii in range(len(data_clean))
                ]

    elif PSA_config == 'cas':
        P_low = 1       
        for factor in adsC_list:
            adsC = 10**factor
            for P_high1 in range(2, 15):  # Loop over high pressure range 1
                for P_high2 in range(2, 15):  # Loop over high pressure range 2
                    data_clean[f'LC_10^{factor}_{PSA_config}_{P_high1}_{P_high2}'] = [
                        Economic(PSA_config, data_clean.iloc[ii, :], adsC, 
                                 data_clean.iloc[ii, :]['Density'], P_high1, P_low, P_high2)[-1] 
                        for ii in range(len(data_clean))
                    ]

# Save the resulting data to a CSV file
data_clean.to_csv('8. All_data.csv')


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44892\542359297.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_clean[f'LC_10^{factor}_{PSA_config}_{P_high1}'] = [Economic(PSA_config,data_clean.iloc[ii, :], adsC, data_clean.iloc[ii, :]['Density'],P_high1, P_low, P_high2)[-1] for ii in range(len(data_clean))]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44892\542359297.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_clean[f'LC_10^{factor}_{PSA_config}_{P_high1}'] = [Economic(PSA_config,data_

### Maximum adsorbent cost (MACO)

#### MACO function

In [48]:
from scipy.interpolate import UnivariateSpline
# Settings 
LC_threshold = 1.75
def FindMaxAds(PSA_config, _target, P_high1, P_low, P_high2):
    # If the adsorbent unit cost is already higher than the threshold at 10 USD/kg, 
    # further calculation is unnecessary.
    if PSA_config == 'PSA':
        LC_10 = _target[f'LC_10^1.0_{PSA_config}_{P_high1}']
    elif PSA_config == 'VSA':
        LC_10 = _target[f'LC_10^1.0_{PSA_config}_{P_low}']
    elif PSA_config == 'cas':
        LC_10 = _target[f'LC_10^1.0_{PSA_config}_{P_high1}_{P_high2}']

    # If the LC value exceeds the threshold or is negative, set a large default value.
    if LC_10 > LC_threshold or LC_10 < 0:
        maxC = 10**10
    
    # If the cost of CH4 can potentially fall below the threshold, perform the calculation.
    else:
        C_dom = np.array([10**adsC for adsC in np.linspace(1, 6, 600)])
        LC_dom = [Economic(PSA_config, _target, adsC, _target['Density'], P_high1, P_low, P_high2)[-1] - LC_threshold for adsC in C_dom]
        fq = UnivariateSpline(C_dom, LC_dom)
        maxC = fq.roots()[0]

    return maxC

# The above function:
# 1. Accepts adsorbent data and reference pressures as input.
# 2. Checks if the threshold is exceeded and evaluates economic feasibility 
#    (based on adsorbent cost).
# 3. Calculates the maximum adsorbent cost.


#### MACO-PSA

In [49]:
# Settings 
LC_threshold = 1.75
PSA_config = 'PSA'
P_high1=8
P_high2=8
P_low = 1

maxC_dict = {}  
for P_high1 in range(2,21): 
    pos_data = data_clean.copy()
    maxC_list = []
    for ii in range(len(pos_data)):
        maxC = FindMaxAds(PSA_config, pos_data.iloc[ii,:], P_high1, P_low, P_high2)
        maxC_list.append(maxC)
    maxC_dict[f'Max_adsC_{PSA_config}_{P_high1}'] = maxC_list

for key, value in maxC_dict.items():
    pos_data[key] = value
pos_data.to_csv(f'{PSA_config}_MACO_LCOM={LC_threshold}.csv')

#### MACO-VSA

In [50]:
# Settings 
LC_threshold = 1.75
PSA_config = 'VSA'
P_high1=8
P_high2=8
P_low = 1

maxC_dict = {}                                      
for P_low in (0.01,0.04,0.07,0.1,0.3,0.5,0.7,0.9,1): 
    pos_data = data_clean.copy()
    maxC_list = []
    for ii in range(len(pos_data)):
        maxC = FindMaxAds(PSA_config, pos_data.iloc[ii,:], P_high1, P_low, P_high2)
        maxC_list.append(maxC)
    maxC_dict[f'Max_adsC_{PSA_config}_{P_low}'] = maxC_list

for key, value in maxC_dict.items():
    pos_data[key] = value
pos_data.to_csv(f'{PSA_config}_MACO_LCOM={LC_threshold}.csv')

#### MACO-Casacde PSA

In [51]:
# Settings 
LC_threshold = 1.75
PSA_config = 'cas'
P_high1=8
P_high2=8
P_low = 1

maxC_dict = {}
for P_high1 in range(2,15):
    for P_high2 in range(2,15):
        pos_data = data_clean.copy()
        maxC_list = []
        for ii in range(len(pos_data)):
            maxC = FindMaxAds(PSA_config, pos_data.iloc[ii,:], P_high1, P_low, P_high2)
            maxC_list.append(maxC)
        maxC_dict[f'Max_adsC_{PSA_config}_{P_high1}_{P_high2}'] = maxC_list

for key, value in maxC_dict.items():
    pos_data[key] = value

pos_data.to_csv(f'{PSA_config}_MACO_LCOM={LC_threshold}.csv')

In [66]:
PSA_MACO = pd.read_csv('PSA_MACO_LCOM=1.75.csv')
VSA_MACO = pd.read_csv('VSA_MACO_LCOM=1.75.csv')
cas_MACO = pd.read_csv('cas_MACO_LCOM=1.75.csv')

# Check for common columns between the three datasets
common_columns = list(set(PSA_MACO.columns) & set(VSA_MACO.columns) & set(cas_MACO.columns))

# Extract unique columns for each dataset
PSA_unique = PSA_MACO.drop(columns=common_columns)
VSA_unique = VSA_MACO.drop(columns=common_columns)
cas_unique = cas_MACO.drop(columns=common_columns)

# Create a common DataFrame (using common columns only once)
common_df = PSA_MACO[common_columns]

# Merge unique columns with the common DataFrame
All_data = pd.concat([common_df, PSA_unique, VSA_unique, cas_unique], axis=1)
All_data.to_csv('8. All_data_LCOM=1.75.csv')
